# Experiments with Tf-Idf

## Overview of Term frequency (tf)
Christian Perone has a nice two part blog post series if you want to get a quick introduction. Part I linked below covers _tf_.
* http://blog.christianperone.com/2011/09/machine-learning-text-feature-extraction-tf-idf-part-i/

Summary
* Tf-Idf measures importance of a word in a document in relation to the corpus. Compare it with Bag of Words which measures presence/absence of a word in a document
* Term frequency (tf) is a counting function representing _number of times_ a word `w` appears in a document

Representation of a document `d` in _tf_ = (tf(word1, d), tf(word2, d) ...). Shape of the _tf vector_ = `number of documents x vocabulary size`

## Data set
We will use simple wiki data from https://github.com/codito/nlp-expt/tree/master/data/simplewiki

Please see the [instructions][] to extract and create `dataset.json`.

[instructions]: https://github.com/codito/nlp-expt/blob/master/data/simplewiki/README.md

In [2]:
import pandas as pd
import numpy as np

# Let's use a simple wiki dataset
with open('../data/simplewiki/dataset.json', 'r', encoding='utf-8') as f:
    df = pd.read_json(f, orient='records', lines=True)

print("Shape: {0}".format(df.shape))
df.head()

Shape: (148714, 5)


,id,url,title,text,categories
0,1,https://simple.wikipedia.org/wiki?curid=1,April,"April\n\nApril is the 4th month of the year, a...",[]
1,2,https://simple.wikipedia.org/wiki?curid=2,August,August\n\nAugust (Aug.) is the 8th month of th...,[]
2,6,https://simple.wikipedia.org/wiki?curid=6,Art,Art\n\n[[]]\n\nArt and crafts is a creative ac...,"[Non-verbal communication, Basic English 850 w..."
3,8,https://simple.wikipedia.org/wiki?curid=8,A,A\n\nA is the first letter of the English alph...,"[Vowel letters, Basic English 850 words, Lette..."
4,9,https://simple.wikipedia.org/wiki?curid=9,Air,Air\n\nAir is the Earth's atmosphere. Air arou...,"[Atmosphere, Physics, Basic English 850 words]"


## Calculate term frequency

`CountVectorizer` from sklearn provides a simple implementation of tf calculation. We are using `spacy` for stop words as we will see later.

In [3]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer

nlp = spacy.load('en_core_web_sm')

# Task: represent articles title as tf vectors
tf_vectorizer = CountVectorizer(dtype=np.double)
vocab_matrix = tf_vectorizer.fit_transform(df.title)

vocab_matrix

<148714x92408 sparse matrix of type '<class 'numpy.float64'>'
	with 335579 stored elements in Compressed Sparse Row format>

Note that we now have a sparse matrix. Let's take a slight detour and learn about sparse matrices.

> In CSR, a matrix is stored as three one-dimensional arrays of row pointers, column indices and values, where the two former are of integer type and the latter of float type, usually double. As the name suggests, non-zero entries are stored per row, where each non-zero is defined by a pair of column index and corresponding value. The column indices and values arrays therefore have a length equal to the total number of non-zero entries. Row indices are given implicitly by the row pointer array, which contains the starting index in the column index and values arrays for the non-zero entries of each row. In other words, the non-zeros for row `i` are at positions `row_ptr[i]` up to but not including `row_ptr[i+1]` in the column index and values arrays. For each row, entries are sorted by column index to allow for faster lookups using a binary search.

![csr format figure](https://op2.github.io/PyOP2/_images/csr.svg)

See https://op2.github.io/PyOP2/linear_algebra.html?highlight=sparse%20matrix#sparse-matrix-storage-formats

Observe that the sparse matrix of tf vectors has `148714` rows and `92408` columns, this is the number of features extracted

## Similarity between vectors

With the vector representation of each Title of wikipedia entries, let's try to compute similarity.

We are using the `cosine` similarity using the `sparse_dot_topn` library. Read about the library here: https://medium.com/wbaa/https-medium-com-ingwbaa-boosting-selection-of-the-most-similar-entities-in-large-scale-datasets-450b3242e618

> Intuition: we have a matrix with rows representing strings and columns the vector representation (call this `vocab_matrix`). We have a second matrix with vector representation of test strings. Measuring similarity between these vectors can be done with dot product.

See http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/

In [4]:
from sparse_dot_topn import awesome_cossim_topn

def compute_cosine_similarity(vocab_matrix, test_matrix):
    # consider top 10 similar strings in vocabulary for every test string
    return awesome_cossim_topn(A=vocab_matrix, B=test_matrix, ntop=10, lower_bound=0.2, n_jobs=8, use_threads=True)
    
def get_similarity(vocab_strings, test_strings, result_matrix):
    rows, cols = result_matrix.nonzero()
    data = [(rows[i], vocab_strings[rows[i]], cols[i], test_strings[cols[i]], result_matrix[rows[i], cols[i]]) for i in range(0, len(rows))]
    return pd.DataFrame(data=data, columns=['Train index', 'Train', 'Test index', 'Test', 'Similarity'])

## Analysis

We'll use above measures to experiment with similarity of wikipedia titles.

### Task 1: Basic similarity

Our `train` and `test` matrix are the same since we are trying to compute similar titles.

In [5]:
# compute duplicates in the vocabulary itself
# test_matrix is just transpose of the trained vocab_matrix
test_matrix = vocab_matrix.transpose()
result_matrix = compute_cosine_similarity(vocab_matrix, test_matrix)

In [6]:
tmp_df = get_similarity(df.title, df.title, result_matrix)

In [7]:
tmp_df[tmp_df['Train index'] != tmp_df['Test index']].sort_values(by=['Similarity'], ascending=False)

,Train index,Train,Test index,Test,Similarity
483031,65973,"The Chronicles of Narnia: The Lion, the Witch ...",92526,The Saga of the Viking Women and Their Voyage ...,19.0
693559,92526,The Saga of the Viking Women and Their Voyage ...,65973,"The Chronicles of Narnia: The Lion, the Witch ...",19.0
576101,77681,List of rivers of the Democratic Republic of t...,77682,List of rivers of the Republic of the Congo,17.0
576110,77682,List of rivers of the Republic of the Congo,77681,List of rivers of the Democratic Republic of t...,17.0
644794,86329,List of Medal of Honor recipients for the Batt...,86328,List of Medal of Honor recipients for the Batt...,16.0
...,...,...,...,...,...
442123,60786,30 (number),80161,30 Rock,1.0
442124,60786,30 (number),141554,Super 30 (movie),1.0
442125,60786,30 (number),143118,February 30,1.0
442126,60786,30 (number),103342,13 Going on 30,1.0


In [8]:
print("train string: {0}, \ntrain vector:\n{1}".format(df.title[65973], vocab_matrix[65973]))
print("test string: {0}, \ntest vector:\n{1}".format(df.title[92526], vocab_matrix[92526]))

train string: The Chronicles of Narnia: The Lion, the Witch and the Wardrobe, 
train vector:
  (0, 60723)	1.0
  (0, 81831)	4.0
  (0, 4983)	1.0
  (0, 49116)	1.0
  (0, 89583)	1.0
  (0, 18225)	1.0
  (0, 58137)	1.0
  (0, 88270)	1.0
test string: The Saga of the Viking Women and Their Voyage to the Waters of the Great Sea Serpent, 
test vector:
  (0, 60723)	2.0
  (0, 81831)	4.0
  (0, 34309)	1.0
  (0, 4983)	1.0
  (0, 73691)	1.0
  (0, 89746)	1.0
  (0, 82743)	1.0
  (0, 86982)	1.0
  (0, 88443)	1.0
  (0, 71826)	1.0
  (0, 74370)	1.0
  (0, 87703)	1.0
  (0, 81854)	1.0


**Analysis**

Our results indicate that strings `The Chronicles of Narnia: The Lion, the Witch and the Wardrobe` and `The Saga of the Viking Women and Their Voyage to the Waters of the Great Sea Serpent`.

There are three same words in both strings:
> of = 60723, The = 81831, and = 4983

Similarity is simple multiplication of both matrix. E.g. (0, 60723) 1x2 + (0, 81831) 4x4 + (0, 4983) 1x1 = 19

### Task 2: Stop words and better term frequency

Let's try two different things and see if the similarity changes.

* Try to take count of occurence of a word in comparison with length of document
* Remove some of the stop words

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_norm_vector = TfidfVectorizer(use_idf=False, stop_words=nlp.Defaults.stop_words)
tf_norm_matrix = tf_norm_vector.fit_transform(df.title)

/usr/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
result_matrix = compute_cosine_similarity(tf_norm_matrix, tf_norm_matrix.transpose())
tmp_df = get_similarity(df.title, df.title, result_matrix)

In [11]:
tmp_df[tmp_df['Train index'] != tmp_df['Test index']].sort_values(by=['Similarity'], ascending=False)

,Train index,Train,Test index,Test,Similarity
461283,63705,List of Latin phrases (A),63720,List of Latin phrases (D),1.000000
208661,30259,Big Brother 9 (UK),88371,Big Brother (UK),1.000000
1112896,146633,2019 FIFA Women's World Cup Group A,146640,2019 FIFA Women's World Cup Group E,1.000000
1112897,146633,2019 FIFA Women's World Cup Group A,146641,2019 FIFA Women's World Cup Group D,1.000000
1112898,146633,2019 FIFA Women's World Cup Group A,146637,2019 FIFA Women's World Cup Group C,1.000000
...,...,...,...,...,...
661932,89448,NH Industries NH90,124105,Kawasaki Heavy Industries &amp; CSR Qingdao Si...,0.204124
1068797,140982,Kaal Bhairav Rahasya 2,130810,"Prithvi Vallabh - Itihaas Bhi, Rahasya Bhi",0.204124
661933,89448,NH Industries NH90,124103,Kawasaki Heavy Industries &amp; CSR Qingdao Si...,0.204124
102937,15177,Szczecin - Dąbie Airstrip,49090,Szczecin-Goleniów &quot;Solidarność&quot; Airport,0.204124


**Analysis**

Results appear much better in this model. Let's look within the first result.

In [74]:
print("Train string: {0}\nTrain matrix:\n{1}".format(df.title[63705], tf_norm_matrix[63705]))
print("Test string: {0}\nTest matrix:\n{1}".format(df.title[63720], tf_norm_matrix[63720]))

Train string: List of Latin phrases (A)
Train matrix:
  (0, 49099)	0.5773502691896258
  (0, 47542)	0.5773502691896258
  (0, 64406)	0.5773502691896258
Test string: List of Latin phrases (D)
Test matrix:
  (0, 49099)	0.5773502691896258
  (0, 47542)	0.5773502691896258
  (0, 64406)	0.5773502691896258


It appears that the individual letters were tokenized and removed :)

In [77]:
tmp_df[tmp_df.Similarity < 0.9999999].sort_values(by=['Similarity'], ascending=False)

,Train index,Train,Test index,Test,Similarity
829696,110596,Sabah Al-Ahmad Al-Jaber Al-Sabah,65421,Jaber al-Ahmad al-Sabah,0.975900
474647,65421,Jaber al-Ahmad al-Sabah,110596,Sabah Al-Ahmad Al-Jaber Al-Sabah,0.975900
466762,64406,Shake,99120,"(Shake, Shake, Shake) Shake Your Booty",0.970143
488743,67240,Mad About You,91071,"It's a Mad, Mad, Mad, Mad World",0.970143
675165,91071,"It's a Mad, Mad, Mad, Mad World",67240,Mad About You,0.970143
...,...,...,...,...,...
487406,67078,Françoise Barré-Sinoussi,103058,"Louise Françoise de Bourbon, Duchess of Bourbon",0.204124
984854,130810,"Prithvi Vallabh - Itihaas Bhi, Rahasya Bhi",140982,Kaal Bhairav Rahasya 2,0.204124
132304,19385,Are You Smarter Than a 5th Grader?,134039,"Michael Brougham, 5th Baron Brougham and Vaux",0.204124
1068797,140982,Kaal Bhairav Rahasya 2,130810,"Prithvi Vallabh - Itihaas Bhi, Rahasya Bhi",0.204124


In [78]:
print("Train string: {0}\nTrain matrix:\n{1}".format(df.title[67240], tf_norm_matrix[67240]))
print("Test string: {0}\nTest matrix:\n{1}".format(df.title[91071], tf_norm_matrix[91071]))

Train string: Mad About You
Train matrix:
  (0, 50812)	1.0
Test string: It's a Mad, Mad, Mad, Mad World
Test matrix:
  (0, 89634)	0.24253562503633297
  (0, 50812)	0.9701425001453319


`Mad About You` is trimmed to a single word `Mad` and second string is all about `Mad`

In [87]:
tmp_df[tmp_df.Similarity < 0.6].sort_values(by=['Similarity'], ascending=False)

,Train index,Train,Test index,Test,Similarity
966460,128414,École nationale supérieure des mines de Paris,123511,École nationale supérieure de formation de l’e...,0.597614
894828,118580,École nationale supérieure de techniques avanc...,123511,École nationale supérieure de formation de l’e...,0.597614
916533,121725,École nationale supérieure de mécanique et d'a...,123511,École nationale supérieure de formation de l’e...,0.597614
930050,123511,École nationale supérieure de formation de l’e...,128414,École nationale supérieure des mines de Paris,0.597614
930051,123511,École nationale supérieure de formation de l’e...,118580,École nationale supérieure de techniques avanc...,0.597614
...,...,...,...,...,...
1068797,140982,Kaal Bhairav Rahasya 2,130810,"Prithvi Vallabh - Itihaas Bhi, Rahasya Bhi",0.204124
102937,15177,Szczecin - Dąbie Airstrip,49090,Szczecin-Goleniów &quot;Solidarność&quot; Airport,0.204124
661933,89448,NH Industries NH90,124103,Kawasaki Heavy Industries &amp; CSR Qingdao Si...,0.204124
984854,130810,"Prithvi Vallabh - Itihaas Bhi, Rahasya Bhi",140982,Kaal Bhairav Rahasya 2,0.204124


In [ ]:
## Test suite for similarity

Spen